Q-5.Uber is a taxi service provider as we know, we need to predict the high booking area using an Unsupervised algorithm and price for the location using a supervised algorithm and use some map function to display the data

Dataset:https://www.kaggle.com/datasets/brllrb/uber-and-lyft-dataset-boston-ma

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import folium
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error

In [2]:
# Load the data
data = pd.read_csv("/content/rideshare_kaggle.csv")
data

,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,9,16,12,2018-12-16 09:30:07,America/New_York,Haymarket Square,North Station,Lyft,...,0.1276,1.544980e+09,39.89,1.545012e+09,43.68,1.544969e+09,33.73,1.545012e+09,38.07,1.544958e+09
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,2,27,11,2018-11-27 02:00:23,America/New_York,Haymarket Square,North Station,Lyft,...,0.1300,1.543252e+09,40.49,1.543234e+09,47.30,1.543252e+09,36.20,1.543291e+09,43.92,1.543252e+09
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,1,28,11,2018-11-28 01:00:22,America/New_York,Haymarket Square,North Station,Lyft,...,0.1064,1.543338e+09,35.36,1.543378e+09,47.55,1.543320e+09,31.04,1.543378e+09,44.12,1.543320e+09
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,4,30,11,2018-11-30 04:53:02,America/New_York,Haymarket Square,North Station,Lyft,...,0.0000,1.543507e+09,34.67,1.543550e+09,45.03,1.543511e+09,30.30,1.543550e+09,38.53,1.543511e+09
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,3,29,11,2018-11-29 03:49:20,America/New_York,Haymarket Square,North Station,Lyft,...,0.0001,1.543421e+09,33.10,1.543403e+09,42.18,1.543421e+09,29.11,1.543392e+09,35.75,1.543421e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,fd3167a5-1ef5-42ad-8100-e060088aa294,1.543568e+09,8,30,11,2018-11-30 08:58:00,America/New_York,North End,North Station,Uber,...,0.0004,1.543594e+09,28.78,1.543579e+09,42.68,1.543601e+09,26.32,1.543576e+09,40.69,1.543612e+09
1958,0135c089-518c-4728-9dd6-7aa51750475c,1.544784e+09,10,14,12,2018-12-14 10:45:05,America/New_York,South Station,Financial District,Uber,...,0.0000,1.544807e+09,27.05,1.544782e+09,46.67,1.544814e+09,24.47,1.544785e+09,43.88,1.544818e+09
1959,34f53769-b730-4f94-a938-a293f1da03de,1.543419e+09,15,28,11,2018-11-28 15:32:08,America/New_York,South Station,Financial District,Uber,...,0.0000,1.543421e+09,33.70,1.543399e+09,42.60,1.543439e+09,29.88,1.543399e+09,36.56,1.543435e+09
1960,4597b0a5-2ecd-402f-af65-e0c3e5dac614,1.543512e+09,17,29,11,2018-11-29 17:12:55,America/New_York,South Station,Financial District,Uber,...,0.0000,1.543511e+09,34.83,1.543550e+09,45.03,1.543511e+09,30.90,1.543550e+09,38.93,1.543511e+09


In [3]:
## Checking Rows & Columns Availabale in Dataset
data.shape

(1962, 57)

In [4]:
## Checking Details Information related with Dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1962 entries, 0 to 1961
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           1962 non-null   object 
 1   timestamp                    1962 non-null   float64
 2   hour                         1962 non-null   int64  
 3   day                          1962 non-null   int64  
 4   month                        1962 non-null   int64  
 5   datetime                     1962 non-null   object 
 6   timezone                     1962 non-null   object 
 7   source                       1962 non-null   object 
 8   destination                  1962 non-null   object 
 9   cab_type                     1962 non-null   object 
 10  product_id                   1962 non-null   object 
 11  name                         1962 non-null   object 
 12  price                        1797 non-null   float64
 13  distance          

In [5]:
## Checking All Columns name present in dataset
data.columns

Index(['id', 'timestamp', 'hour', 'day', 'month', 'datetime', 'timezone',
       'source', 'destination', 'cab_type', 'product_id', 'name', 'price',
       'distance', 'surge_multiplier', 'latitude', 'longitude', 'temperature',
       'apparentTemperature', 'short_summary', 'long_summary',
       'precipIntensity', 'precipProbability', 'humidity', 'windSpeed',
       'windGust', 'windGustTime', 'visibility', 'temperatureHigh',
       'temperatureHighTime', 'temperatureLow', 'temperatureLowTime',
       'apparentTemperatureHigh', 'apparentTemperatureHighTime',
       'apparentTemperatureLow', 'apparentTemperatureLowTime', 'icon',
       'dewPoint', 'pressure', 'windBearing', 'cloudCover', 'uvIndex',
       'visibility.1', 'ozone', 'sunriseTime', 'sunsetTime', 'moonPhase',
       'precipIntensityMax', 'uvIndexTime', 'temperatureMin',
       'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime',
       'apparentTemperatureMin', 'apparentTemperatureMinTime',
       'apparentTemperat

In [53]:
data=data.dropna()
data

,id,timestamp,hour,day,month,datetime,timezone,source,destination,cab_type,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,1.544953e+09,9,16,12,2018-12-16 09:30:07,America/New_York,Haymarket Square,North Station,Lyft,...,0.1276,1.544980e+09,39.89,1.545012e+09,43.68,1.544969e+09,33.73,1.545012e+09,38.07,1.544958e+09
1,4bd23055-6827-41c6-b23b-3c491f24e74d,1.543284e+09,2,27,11,2018-11-27 02:00:23,America/New_York,Haymarket Square,North Station,Lyft,...,0.1300,1.543252e+09,40.49,1.543234e+09,47.30,1.543252e+09,36.20,1.543291e+09,43.92,1.543252e+09
2,981a3613-77af-4620-a42a-0c0866077d1e,1.543367e+09,1,28,11,2018-11-28 01:00:22,America/New_York,Haymarket Square,North Station,Lyft,...,0.1064,1.543338e+09,35.36,1.543378e+09,47.55,1.543320e+09,31.04,1.543378e+09,44.12,1.543320e+09
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,1.543554e+09,4,30,11,2018-11-30 04:53:02,America/New_York,Haymarket Square,North Station,Lyft,...,0.0000,1.543507e+09,34.67,1.543550e+09,45.03,1.543511e+09,30.30,1.543550e+09,38.53,1.543511e+09
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,1.543463e+09,3,29,11,2018-11-29 03:49:20,America/New_York,Haymarket Square,North Station,Lyft,...,0.0001,1.543421e+09,33.10,1.543403e+09,42.18,1.543421e+09,29.11,1.543392e+09,35.75,1.543421e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,e23fbf8e-35ed-460a-946f-13b986ff46f1,1.544809e+09,17,14,12,2018-12-14 17:40:15,America/New_York,Beacon Hill,Northeastern University,Uber,...,0.0000,1.544807e+09,26.93,1.544789e+09,46.68,1.544814e+09,27.58,1.544789e+09,43.98,1.544818e+09
1957,fd3167a5-1ef5-42ad-8100-e060088aa294,1.543568e+09,8,30,11,2018-11-30 08:58:00,America/New_York,North End,North Station,Uber,...,0.0004,1.543594e+09,28.78,1.543579e+09,42.68,1.543601e+09,26.32,1.543576e+09,40.69,1.543612e+09
1958,0135c089-518c-4728-9dd6-7aa51750475c,1.544784e+09,10,14,12,2018-12-14 10:45:05,America/New_York,South Station,Financial District,Uber,...,0.0000,1.544807e+09,27.05,1.544782e+09,46.67,1.544814e+09,24.47,1.544785e+09,43.88,1.544818e+09
1959,34f53769-b730-4f94-a938-a293f1da03de,1.543419e+09,15,28,11,2018-11-28 15:32:08,America/New_York,South Station,Financial District,Uber,...,0.0000,1.543421e+09,33.70,1.543399e+09,42.60,1.543439e+09,29.88,1.543399e+09,36.56,1.543435e+09


In [7]:
 data.duplicated().sum()

0

In [8]:
## segregate numerical and categorical columns

numerical_columns=data.columns[data.dtypes!='object']
categorical_columns=data.columns[data.dtypes=='object']
print("Numerical columns:",numerical_columns)
print('Categorical Columns:',categorical_columns)

Numerical columns: Index(['timestamp', 'hour', 'day', 'month', 'price', 'distance',
       'surge_multiplier', 'latitude', 'longitude', 'temperature',
       'apparentTemperature', 'precipIntensity', 'precipProbability',
       'humidity', 'windSpeed', 'windGust', 'windGustTime', 'visibility',
       'temperatureHigh', 'temperatureHighTime', 'temperatureLow',
       'temperatureLowTime', 'apparentTemperatureHigh',
       'apparentTemperatureHighTime', 'apparentTemperatureLow',
       'apparentTemperatureLowTime', 'dewPoint', 'pressure', 'windBearing',
       'cloudCover', 'uvIndex', 'visibility.1', 'ozone', 'sunriseTime',
       'sunsetTime', 'moonPhase', 'precipIntensityMax', 'uvIndexTime',
       'temperatureMin', 'temperatureMinTime', 'temperatureMax',
       'temperatureMaxTime', 'apparentTemperatureMin',
       'apparentTemperatureMinTime', 'apparentTemperatureMax',
       'apparentTemperatureMaxTime'],
      dtype='object')
Categorical Columns: Index(['id', 'datetime', 'timezone'

In [9]:
columns_to_encode = ['id', 'datetime', 'timezone', 'source', 'destination', 'cab_type',
       'product_id', 'name', 'short_summary', 'long_summary', 'icon']


In [10]:
encoder = OneHotEncoder(sparse=False)
encoded_cols = encoder.fit_transform(data[columns_to_encode])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(columns_to_encode))
encoded_df

,id_001e4903-c2ec-4752-8830-206149cfb440,id_0040ca98-3a68-47e4-a3a0-234176042f19,id_00497724-383e-4d2a-90be-854294c27dfd,id_0091d65d-8904-4034-91d8-6deb8737a814,id_009e9c53-074d-43cf-aef2-0fbc7a47ed3d,id_00ac7cd7-8fc9-40a0-87de-abfb43b0581f,id_00dc269c-2990-402b-b834-9fe0cede0bae,id_00dd58fc-fa9e-4eeb-bb6e-dfeb94611e85,id_00e45042-50ed-4683-b213-14967ab2f873,id_00ffaa21-d3ce-40f6-bf8f-67eb843fca97,...,long_summary_ Possible drizzle in the morning.,long_summary_ Rain throughout the day.,"long_summary_ Rain until morning, starting again in the evening.",icon_ clear-day,icon_ clear-night,icon_ cloudy,icon_ fog,icon_ partly-cloudy-day,icon_ partly-cloudy-night,icon_ rain
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1794,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [14]:
# Drop columns 
columns_to_drop = ['id', 'datetime', 'timezone', 'source', 'destination', 'cab_type',
       'product_id', 'name', 'short_summary', 'long_summary', 'icon']
df3= data.drop(columns_to_drop, axis=1)
df3

,timestamp,hour,day,month,price,distance,surge_multiplier,latitude,longitude,temperature,...,precipIntensityMax,uvIndexTime,temperatureMin,temperatureMinTime,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime
0,1.544953e+09,9,16,12,5.0,0.44,1.0,42.2148,-71.0330,42.34,...,0.1276,1.544980e+09,39.89,1.545012e+09,43.68,1.544969e+09,33.73,1.545012e+09,38.07,1.544958e+09
1,1.543284e+09,2,27,11,11.0,0.44,1.0,42.2148,-71.0330,43.58,...,0.1300,1.543252e+09,40.49,1.543234e+09,47.30,1.543252e+09,36.20,1.543291e+09,43.92,1.543252e+09
2,1.543367e+09,1,28,11,7.0,0.44,1.0,42.2148,-71.0330,38.33,...,0.1064,1.543338e+09,35.36,1.543378e+09,47.55,1.543320e+09,31.04,1.543378e+09,44.12,1.543320e+09
3,1.543554e+09,4,30,11,26.0,0.44,1.0,42.2148,-71.0330,34.38,...,0.0000,1.543507e+09,34.67,1.543550e+09,45.03,1.543511e+09,30.30,1.543550e+09,38.53,1.543511e+09
4,1.543463e+09,3,29,11,9.0,0.44,1.0,42.2148,-71.0330,37.44,...,0.0001,1.543421e+09,33.10,1.543403e+09,42.18,1.543421e+09,29.11,1.543392e+09,35.75,1.543421e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,1.544809e+09,17,14,12,25.0,1.59,1.0,42.3503,-71.0810,44.89,...,0.0000,1.544807e+09,26.93,1.544789e+09,46.68,1.544814e+09,27.58,1.544789e+09,43.98,1.544818e+09
1957,1.543568e+09,8,30,11,7.0,0.30,1.0,42.3519,-71.0551,30.09,...,0.0004,1.543594e+09,28.78,1.543579e+09,42.68,1.543601e+09,26.32,1.543576e+09,40.69,1.543612e+09
1958,1.544784e+09,10,14,12,6.5,0.63,1.0,42.3519,-71.0643,27.71,...,0.0000,1.544807e+09,27.05,1.544782e+09,46.67,1.544814e+09,24.47,1.544785e+09,43.88,1.544818e+09
1959,1.543419e+09,15,28,11,26.5,0.63,1.0,42.3519,-71.0551,41.24,...,0.0000,1.543421e+09,33.70,1.543399e+09,42.60,1.543439e+09,29.88,1.543399e+09,36.56,1.543435e+09


In [42]:
# Concatenate the DataFrames vertically
df4 = pd.concat([df3,encoded_df])

df4


,timestamp,hour,day,month,price,distance,surge_multiplier,latitude,longitude,temperature,...,long_summary_ Possible drizzle in the morning.,long_summary_ Rain throughout the day.,"long_summary_ Rain until morning, starting again in the evening.",icon_ clear-day,icon_ clear-night,icon_ cloudy,icon_ fog,icon_ partly-cloudy-day,icon_ partly-cloudy-night,icon_ rain
0,1.544953e+09,9.0,16.0,12.0,5.0,0.44,1.0,42.2148,-71.033,42.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.543284e+09,2.0,27.0,11.0,11.0,0.44,1.0,42.2148,-71.033,43.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.543367e+09,1.0,28.0,11.0,7.0,0.44,1.0,42.2148,-71.033,38.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.543554e+09,4.0,30.0,11.0,26.0,0.44,1.0,42.2148,-71.033,34.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.543463e+09,3.0,29.0,11.0,9.0,0.44,1.0,42.2148,-71.033,37.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [78]:
# Replace missing values with the mean
mean_values = df4.mean()
df5 = df4.fillna(mean_values)
df5

,timestamp,hour,day,month,price,distance,surge_multiplier,latitude,longitude,temperature,...,long_summary_ Possible drizzle in the morning.,long_summary_ Rain throughout the day.,"long_summary_ Rain until morning, starting again in the evening.",icon_ clear-day,icon_ clear-night,icon_ cloudy,icon_ fog,icon_ partly-cloudy-day,icon_ partly-cloudy-night,icon_ rain
0,1.544953e+09,9.00000,16.000000,12.000000,5.000000,0.440000,1.000000,42.214800,-71.033000,42.34000,...,0.002784,0.073497,0.075167,0.033408,0.097996,0.297884,0.016147,0.1598,0.235523,0.159243
1,1.543284e+09,2.00000,27.000000,11.000000,11.000000,0.440000,1.000000,42.214800,-71.033000,43.58000,...,0.002784,0.073497,0.075167,0.033408,0.097996,0.297884,0.016147,0.1598,0.235523,0.159243
2,1.543367e+09,1.00000,28.000000,11.000000,7.000000,0.440000,1.000000,42.214800,-71.033000,38.33000,...,0.002784,0.073497,0.075167,0.033408,0.097996,0.297884,0.016147,0.1598,0.235523,0.159243
3,1.543554e+09,4.00000,30.000000,11.000000,26.000000,0.440000,1.000000,42.214800,-71.033000,34.38000,...,0.002784,0.073497,0.075167,0.033408,0.097996,0.297884,0.016147,0.1598,0.235523,0.159243
4,1.543463e+09,3.00000,29.000000,11.000000,9.000000,0.440000,1.000000,42.214800,-71.033000,37.44000,...,0.002784,0.073497,0.075167,0.033408,0.097996,0.297884,0.016147,0.1598,0.235523,0.159243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,1.544025e+09,11.50167,18.222717,11.564588,16.343541,2.118285,1.009605,42.336717,-71.066069,39.62931,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000
1792,1.544025e+09,11.50167,18.222717,11.564588,16.343541,2.118285,1.009605,42.336717,-71.066069,39.62931,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0000,0.000000,0.000000
1793,1.544025e+09,11.50167,18.222717,11.564588,16.343541,2.118285,1.009605,42.336717,-71.066069,39.62931,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,1.000000,0.000000
1794,1.544025e+09,11.50167,18.222717,11.564588,16.343541,2.118285,1.009605,42.336717,-71.066069,39.62931,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0000,0.000000,0.000000


In [85]:
# Extract coordinates
coordinates = df5[['latitude', 'longitude']]

# Apply K-means clustering
kmeans = KMeans(n_clusters=3)  # Set the desired number of clusters
kmeans.fit(coordinates)

# Assign cluster labels to the data
df5['cluster_label'] = kmeans.labels_


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [96]:
import folium


In [97]:
# Assuming you have latitude and longitude information for the cluster centers
center_latitudes = [42.214800, 42.214800, 42.214800]  # Replace lat1, lat2, lat3 with actual values
center_longitudes = [-71.033000, -71.033000, -71.033000]  # Replace lon1, lon2, lon3 with actual values

# Create a map centered around a specific location (e.g., the first cluster center)
map_clusters = folium.Map(location=[center_latitudes[0], center_longitudes[0]], zoom_start=12)


In [98]:
# Iterate over the clusters and add markers to the map
for i in range(len(center_latitudes)):
    folium.Marker(
        location=[center_latitudes[i], center_longitudes[i]],
        popup=f'Cluster {i+1}',
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_clusters)


In [100]:
# Save the map as an HTML file
map_clusters.save('cluster_map.html')

# Display the map
map_clusters

In [89]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df5[['latitude','longitude']], df5['price'], test_size=0.2)

# Create and train the linear regression model
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

# Predict prices for the test set
y_pred = regression_model.predict(X_test)



In [93]:
# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 42.70109357676387
